In [1]:
import pandas as pd

# Load datasets (adjust file paths if needed)
dmsp_df = pd.read_csv('/Users/bishmaybarik/Library/CloudStorage/OneDrive-ShivNadarInstitutionofEminence/nightlight_atlas/01_data/01_raw/shrug-dmsp-csv/dmsp_shrid.csv')
viirs_df = pd.read_csv('/Users/bishmaybarik/Library/CloudStorage/OneDrive-ShivNadarInstitutionofEminence/nightlight_atlas/01_data/01_raw/shrug-viirs-annual-csv/viirs_annual_shrid.csv')
rural_df = pd.read_csv('/Users/bishmaybarik/Library/CloudStorage/OneDrive-ShivNadarInstitutionofEminence/nightlight_atlas/01_data/01_raw/shrug-secc-cons-rural-csv/secc_cons_rural_shrid.csv')
urban_df = pd.read_csv('/Users/bishmaybarik/Library/CloudStorage/OneDrive-ShivNadarInstitutionofEminence/nightlight_atlas/01_data/01_raw/shrug-secc-cons-urban-csv/secc_cons_urban_shrid.csv')

# 1. Checking duplicates
def check_duplicates(df, id_columns, dataset_name):
    duplicates = df.duplicated(subset=id_columns).sum()
    print(f"{dataset_name}: {duplicates} duplicate rows found based on {id_columns}")

check_duplicates(dmsp_df, ['shrid2', 'year', 'dmsp_f_version'], "DMSP")
check_duplicates(viirs_df, ['shrid2', 'year', 'category'], "VIIRS")
check_duplicates(rural_df, ['shrid2'], "Rural Consumption")
check_duplicates(urban_df, ['shrid2'], "Urban Consumption")

# 2. Standardizing `shrid2`
datasets = {'DMSP': dmsp_df, 'VIIRS': viirs_df, 'Rural Consumption': rural_df, 'Urban Consumption': urban_df}

for name, df in datasets.items():
    df['shrid2'] = df['shrid2'].str.strip().str.lower()
    print(f"{name}: Standardized `shrid2`")

# 3. Examining year ranges
if 'year' in dmsp_df.columns:
    print(f"DMSP Year Range: {dmsp_df['year'].min()} - {dmsp_df['year'].max()}")
if 'year' in viirs_df.columns:
    print(f"VIIRS Year Range: {viirs_df['year'].min()} - {viirs_df['year'].max()}")

# Summarizing dataframes
print("Rural Dataset Year Range: SECC does not have a `year` column (cross-sectional).")
print("Urban Dataset Year Range: SECC does not have a `year` column (cross-sectional).")


DMSP: 0 duplicate rows found based on ['shrid2', 'year', 'dmsp_f_version']
VIIRS: 0 duplicate rows found based on ['shrid2', 'year', 'category']
Rural Consumption: 0 duplicate rows found based on ['shrid2']
Urban Consumption: 0 duplicate rows found based on ['shrid2']
DMSP: Standardized `shrid2`
VIIRS: Standardized `shrid2`
Rural Consumption: Standardized `shrid2`
Urban Consumption: Standardized `shrid2`
DMSP Year Range: 1994 - 2013
VIIRS Year Range: 2012 - 2023
Rural Dataset Year Range: SECC does not have a `year` column (cross-sectional).
Urban Dataset Year Range: SECC does not have a `year` column (cross-sectional).


In [2]:
# Subset DMSP data for 2011
dmsp_2011 = dmsp_df[dmsp_df['year'] == 2011]

# Merge DMSP with SECC rural and urban datasets
rural_merged = pd.merge(rural_df, dmsp_2011, on='shrid2', how='inner')
urban_merged = pd.merge(urban_df, dmsp_2011, on='shrid2', how='inner')

# Combine rural and urban datasets for comprehensive analysis
secc_combined = pd.concat([
    rural_merged.assign(area_type='rural'), 
    urban_merged.assign(area_type='urban')
], ignore_index=True)

# Verify the merge
print("Merged Dataset Summary:")
print(secc_combined.info())

# Save the combined dataset to the specified path
output_path = "/Users/bishmaybarik/Library/CloudStorage/OneDrive-ShivNadarInstitutionofEminence/nightlight_atlas/01_data/02_processed/secc_combined.csv"
secc_combined.to_csv(output_path, index=False)
print(f"Dataset saved to {output_path}")


Merged Dataset Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567429 entries, 0 to 567428
Data columns (total 22 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   shrid2                          567429 non-null  object 
 1   secc_cons_rural                 563741 non-null  float64
 2   secc_cons_pc_rural              563741 non-null  float64
 3   secc_pov_rate_rural             563741 non-null  float64
 4   secc_pov_rate_tend_rural        563741 non-null  float64
 5   _mean_p_miss                    567429 non-null  float64
 6   _core_p_miss                    567429 non-null  float64
 7   _target_weight_share            567429 non-null  float64
 8   _target_group_max_weight_share  567429 non-null  float64
 9   dmsp_max_light                  567429 non-null  int64  
 10  dmsp_total_light                567429 non-null  int64  
 11  dmsp_total_light_cal            567429 non-null  float

In [3]:
# Creating new variables by combining rural and urban values
secc_combined['secc_cons'] = secc_combined['secc_cons_rural'].combine_first(secc_combined['secc_cons_urban'])
secc_combined['secc_cons_pc'] = secc_combined['secc_cons_pc_rural'].combine_first(secc_combined['secc_cons_pc_urban'])
secc_combined['secc_pov_rate'] = secc_combined['secc_pov_rate_rural'].combine_first(secc_combined['secc_pov_rate_urban'])
secc_combined['secc_pov_rate_tend'] = secc_combined['secc_pov_rate_tend_rural'].combine_first(secc_combined['secc_pov_rate_tend_urban'])

# Dropping the original rural and urban variables
columns_to_drop = [
    'secc_cons_rural', 'secc_cons_pc_rural', 'secc_pov_rate_rural', 'secc_pov_rate_tend_rural',
    'secc_cons_urban', 'secc_cons_pc_urban', 'secc_pov_rate_urban', 'secc_pov_rate_tend_urban'
]
secc_combined.drop(columns=columns_to_drop, inplace=True)

# Verify the updated dataset
print("Updated Dataset Summary:")
print(secc_combined.info())

# Saving the updated dataset
updated_output_path = "/Users/bishmaybarik/Library/CloudStorage/OneDrive-ShivNadarInstitutionofEminence/nightlight_atlas/01_data/02_processed/secc_combined_updated.csv"
secc_combined.to_csv(updated_output_path, index=False)
print(f"Updated dataset saved to {updated_output_path}")


Updated Dataset Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 567429 entries, 0 to 567428
Data columns (total 18 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   shrid2                          567429 non-null  object 
 1   _mean_p_miss                    567429 non-null  float64
 2   _core_p_miss                    567429 non-null  float64
 3   _target_weight_share            567429 non-null  float64
 4   _target_group_max_weight_share  567429 non-null  float64
 5   dmsp_max_light                  567429 non-null  int64  
 6   dmsp_total_light                567429 non-null  int64  
 7   dmsp_total_light_cal            567429 non-null  float64
 8   dmsp_mean_light                 567429 non-null  float64
 9   dmsp_mean_light_cal             567429 non-null  float64
 10  dmsp_num_cells                  567429 non-null  int64  
 11  dmsp_f_version                  567429 non-null  obje